In [11]:
from PIL import Image, ImageFile
import os
from concurrent.futures import ThreadPoolExecutor

In [12]:
# Aumenta il limite per la dimensione delle immagini per evitare DecompressionBombError
Image.MAX_IMAGE_PIXELS = None
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [13]:
# Directory contenente le immagini
image_dir = r"C:\Users\davide.biganzoli\OneDrive - Università degli Studi di Milano\Lavoro\2024\Methylation\207127940135"

# Lista dei file immagine
image_files = [file for file in os.listdir(image_dir) if file.endswith(('jpg', 'jpeg', 'png'))]

In [14]:
def get_t_value(filename):
    # Estrai il valore di T dal nome del file
    base_name = os.path.basename(filename)
    t_index = base_name.find('_T') + 2
    t_value = int(base_name[t_index:t_index+2])  # Assumiamo che T sia seguito da due cifre
    return t_value

def get_num_value(filename):
    # Estrai il valore del numero verticale dal nome del file
    base_name = os.path.basename(filename)
    num_part = base_name.split('_')[2]
    num_value = int(num_part.split('-')[0])
    return num_value

In [15]:
def create_mosaic(image_list, output_path, vertical_shift_percent=0.5, horizontal_shift_percent=0.048):
    # Ordina le immagini in base al nome per mantenere l'ordine
    image_list.sort(key=lambda x: (get_num_value(x), get_t_value(x)))

    # Apri tutte le immagini
    images = []
    for img_file in image_list:
        try:
            img = Image.open(os.path.join(image_dir, img_file))
            img.verify()  # Verifica l'integrità dell'immagine
            img = Image.open(os.path.join(image_dir, img_file))  # Ri-apri l'immagine dopo verifica
            img.load()  # Carica i dati dell'immagine
            images.append(img)
        except (IOError, SyntaxError) as e:
            print(f"Immagine {img_file} corrotta o non valida: {e}")
            continue

    # Assumi che tutte le immagini abbiano la stessa dimensione
    if not images:
        print("Nessuna immagine valida trovata.")
        return

    img_width, img_height = images[0].size
    vertical_overlap = int(img_height * vertical_shift_percent)  # Sovrapposizione verticale personalizzata
    horizontal_overlap = int(img_width * horizontal_shift_percent)  # Sovrapposizione orizzontale personalizzata

    # Determina il numero di righe e colonne
    num_rows = len(set(get_num_value(img) for img in image_list))
    num_cols = 13  # Numero di colonne per ogni shift di T

    # Crea una nuova immagine grande abbastanza per contenere tutte le altre
    mosaic_width = img_width * num_cols - horizontal_overlap * (num_cols - 1)
    mosaic_height = img_height * num_rows - vertical_overlap * (num_rows - 1)
    mosaic_image = Image.new('RGB', (mosaic_width, mosaic_height))

    # Aggiungi ogni immagine nel mosaico
    for img, file in zip(images, image_list):
        t_value = get_t_value(file) - 1  # T01 corrisponde alla colonna 0
        num_value = get_num_value(file) - 1  # Numero corrisponde allo spostamento verticale
        x = t_value * (img_width - horizontal_overlap)
        y = num_value * (img_height - vertical_overlap)  # Aggiungi sovrapposizione
        mosaic_image.paste(img, (x, y))

    # Salva il mosaico
    mosaic_image.save(output_path, format='PNG')

In [16]:
def process_sample(prefix, vertical_shift_percent=0.5, horizontal_shift_percent=0.048):
    sample_images = [file for file in image_files if prefix in file and 'green' in file]
    mosaic_path = os.path.join(image_dir, f"{prefix}_mosaic.png")
    create_mosaic(sample_images, mosaic_path, vertical_shift_percent, horizontal_shift_percent)
    return mosaic_path

In [17]:
def create_final_chip(image_dir, output_path, vertical_shift_percent=0.5, horizontal_shift_percent=0.048):
    sample_prefixes = [f"R0{i}C01" for i in range(1, 9)]
    mosaic_paths = []

    # Utilizza ThreadPoolExecutor per parallelizzare la creazione dei mosaici
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_sample, prefix, vertical_shift_percent, horizontal_shift_percent) for prefix in sample_prefixes]
        for future in futures:
            mosaic_paths.append(future.result())

    # Carica tutti i mosaici
    mosaics = [Image.open(mosaic) for mosaic in mosaic_paths]

    # Assumi che tutti i mosaici abbiano la stessa larghezza
    mosaic_width = mosaics[0].width
    mosaic_height = mosaics[0].height

    # Crea una nuova immagine per il chip finale
    final_chip_height = mosaic_height * len(mosaics)
    final_chip_image = Image.new('RGB', (mosaic_width, final_chip_height))

    # Aggiungi ogni mosaico al chip finale
    for i, mosaic in enumerate(mosaics):
        final_chip_image.paste(mosaic, (0, i * mosaic_height))

    # Salva il chip finale
    final_chip_image.save(output_path, format='PNG')


In [18]:
# Crea il chip finale unendo tutti i mosaici dei campioni
create_final_chip(image_dir, r"C:\Users\davide.biganzoli\OneDrive - Università degli Studi di Milano\Lavoro\2024\Methylation\final_chip.png", vertical_shift_percent=0.5, horizontal_shift_percent=0.048)